# Conformación dataset

In [ ]:
import glob
import xarray as xr
import geopandas as gpd
import googlemaps
import numpy as np

In [ ]:
# Ruta al archivo shape
shapefile_path = "../shapes/shape_boyaca_municipios.shp"

# Carpeta que contiene los archivos .nc (Descargando usando el notebook 1.get_data.ipynb)
nc_folder_path = "../data/archivos_climate_serv/"

# Cargar el archivo shape
gdf = gpd.read_file(shapefile_path)

# Obtener las coordenadas extremas iniciales
min_latitude = float('inf')
max_latitude = float('-inf')
min_longitude = float('inf')
max_longitude = float('-inf')

# Iterar sobre los polígonos del archivo shape
for i, geometry in enumerate(gdf.geometry):
    # Obtener los límites de latitud y longitud del polígono actual
    polygon_bounds = geometry.bounds
    
    # Actualizar los valores máximos y mínimos de latitud y longitud
    min_latitude = min(min_latitude, polygon_bounds[1])
    max_latitude = max(max_latitude, polygon_bounds[3])
    min_longitude = min(min_longitude, polygon_bounds[0])
    max_longitude = max(max_longitude, polygon_bounds[2])

min_longitude -= 0.001
max_longitude += 0.001

# Crear una lista para almacenar los datos filtrados
filtered_datasets = []

# Obtener la lista de archivos .nc en la carpeta
nc_file_paths = glob.glob(nc_folder_path + "*.nc")

# Iterar sobre los archivos .nc
for nc_file_path in nc_file_paths:
    # Leer el archivo .nc utilizando xarray
    ds = xr.open_dataset(nc_file_path)
    
    # Filtrar por latitud y longitud máximas y mínimas
    ds_filtered = ds.sel(
        longitude=slice(min_longitude, max_longitude),
        latitude=slice(min_latitude, max_latitude)
    )
    
    # Agregar el dataset filtrado a la lista
    filtered_datasets.append(ds_filtered)

# Combinar los datasets filtrados en uno solo
combined_dataset = xr.concat(filtered_datasets, dim="time")

# Ruta para guardar el archivo .nc combinado
output_nc_path = "../data/data_filtered.nc"

# Guardar el dataset combinado en un nuevo archivo .nc
combined_dataset.to_netcdf(output_nc_path)

In [ ]:
# Cargar el archivo .nc utilizando xarray
data_filtered = xr.open_dataset('../data/data_filtered.nc')

In [ ]:
# Crear un dataframe a partir del archivo .nc
df_filtered = data_filtered.to_dataframe().reset_index()

In [ ]:
# Obtener los pares de latitud y longitud únicos
lat_lon = df_filtered[['latitude', 'longitude']].drop_duplicates()

In [ ]:
# Crea una instancia del cliente de Google Maps
gmaps = googlemaps.Client(key='API_KEY')

# Define una función para obtener el municipio y departamento a partir de las coordenadas
def get_municipality_department(lat, lng):
    # Realiza la geocodificación inversa
    reverse_geocode_result = gmaps.reverse_geocode((lat, lng))
    
    # Busca el municipio y departamento en los resultados
    municipality = None
    department = None
    for result in reverse_geocode_result:
        for component in result['address_components']:
            if 'administrative_area_level_2' in component['types']:
                municipality = component['long_name']
            elif 'administrative_area_level_1' in component['types']:
                department = component['long_name']
    
    return municipality, department

# Obtener los pares de latitud y longitud únicos
lat_lon_unique = lat_lon.drop_duplicates()

# Crea listas vacías para almacenar los municipios y departamentos
municipalities = []
departments = []

# Itera sobre los pares de latitud y longitud únicos y obtiene el municipio y departamento correspondientes
for index, row in lat_lon_unique.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    municipality, department = get_municipality_department(lat, lon)
    municipalities.append(municipality)
    departments.append(department)

# Agrega las columnas de municipio y departamento al DataFrame
lat_lon_unique['municipality'] = municipalities
lat_lon_unique['department'] = departments

In [ ]:
# Hacer un merge entre el DataFrame original y el DataFrame con los municipios y departamentos
df_filtered_merge = df_filtered.merge(lat_lon_unique, on=['latitude', 'longitude'])

In [ ]:
# Filtrar por departamento Boyacá
df_boyaca = df_filtered_merge[df_filtered_merge['department'] == 'Boyaca']

In [ ]:
# reset index
df_boyaca = df_boyaca.reset_index(drop=True)
# Eliminar columna de departamento
df_boyaca = df_boyaca.drop(columns=['department'])

In [ ]:
# Exportar el DataFrame a un archivo .nc
ds_boyaca = df_boyaca.to_xarray()
ds_boyaca.to_netcdf('../data/data_boyaca_final.nc')